## Rabi (§TYPE§)
### §PULSE_LENGTH§ns, pi in §PULSE_AMP§V

In [ ]:
#f.close()
f  = h5py.File(§HDF5_PATH§, 'r', swmr=True)
pa = f['parameters']
da = f['data']
#f.close()

amp   = da['amp'][()]
data1 = da['I1'][()] + 1j * da['Q1'][()]
data2 = da['I2'][()] + 1j * da['Q2'][()] 
data  = IQ_rot((data1 - data2))
amp2  = np.linspace(amp[0], amp[-1], 1000)
limit = §LINEARITY_AMP_LIMIT§

def cosine(t, a_rabi, b, c):
    t_arr = np.array(t)
    return b*np.cos(2*np.pi*t_arr/a_rabi)+c

# discard non-linear part (§LINEARITY_AMP_LIMIT§ is defined in assumptions.py)
Nstop = np.amin(np.where(amp >= limit))

# estimate a_rabi
rfft = np.abs(np.fft.rfft(data.real - np.mean(data.real)))
freq = np.fft.rfftfreq(data.real.size, d=(amp[-1]-amp[0])/(amp.size-1))
rabi = 1/freq[np.argmax(rfft)]
print(f'a_rabi ~ {rabi}')

# plot FFT
#fig, ax = plt.subplots()
#ax.plot(freq[:freq.size//2], rfft[:freq.size//2])
#ax.axvline(1/rabi, c='black', ls='--')
#ax.legend(('FFT(data)', f'1/a_rabi = {1/rabi:f}\na_rabi = {rabi:f}'))

popt, pcov = opt.curve_fit(cosine, amp[:Nstop], np.real(data[:Nstop]), (rabi, 0, 0))

print(f'a_rabi, b, c = {popt}')

fig, ax = plt.subplots()

ax.plot(amp, data.real, '.-', label='Data')
ax.plot(amp2, cosine(amp2, *popt), 'r--')
ax.plot(amp2[amp2 <= limit], cosine(amp2[amp2 <= limit], *popt), 'r-', label=f'Fit: a_rabi = {popt[0]:f}')
ax.legend()

_opt = popt
_cov = pcov
_result = {'a_rabi': popt[0]}
_result